In [ ]:
import json
import os
from io import BytesIO

from documentcloud import DocumentCloud
from PIL import Image

import pytesseract # also import pytesseract to extract text using tesseract (the text output from documentcloud does not work well)

In [ ]:
client = DocumentCloud(username="Your MuckRock ID", password="Your Password")

In [ ]:
DocumentID = 6183219 # This document https://www.documentcloud.org/documents/6183219-BakersfieldPoliceDeptAaronStringer08062014OISCaseF

In [ ]:
document = client.documents.get(DocumentID)

In [ ]:
# Get the third annotation
note = document.annotations[1]

In [ ]:
# this is the URL where the text position JSON is located
positions_url = (
    f"{document.asset_url}documents/{document.id}/pages/"
    f"{document.slug}-p{note.page + 1}.position.json"
)

In [ ]:
# fetch the text position JSON -- This line produces an error
text_positions = json.loads(document._get_url(positions_url, True))

In [ ]:
# this is looking for all words which are entirely contained within the note
# you could change the inequalities here if you prefer to include words
# which are partially contained within the note
text = " ".join(
    p["text"]
    for p in text_positions
    if p["x1"] > note.x1
    and p["x2"] < note.x2
    and p["y1"] > note.y1
    and p["y2"] < note.y2
)


In [ ]:
# fetch the image for the page
image = Image.open(BytesIO(document.get_image(note.page + 1)))
# crop the image
# the note coordinates are in percentages,
# so multiply by the image width or height
image = image.crop(
    (
        image.width * note.x1,
        image.height * note.y1,
        image.width * note.x2,
        image.height * note.y2,
    )
)

In [ ]:
print(pytesseract.image_to_string(image))